In [96]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bkcharts import Bar, Scatter
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import HoverTool
from collections import OrderedDict
import copy
import pandas as pd
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [97]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [98]:
# Read in new PUF and associated data
gf = pd.read_csv('../taxdata/stage1/growfactors.csv')
wt = pd.read_csv('../taxdata/puf_stage2/puf_weights.csv')
puf = pd.read_csv('../taxdata/puf_data/puf.csv')
adj = pd.read_csv('../taxdata/puf_stage3/puf_ratios.csv', index_col=0)
adj = adj.transpose()

In [99]:
# base calculator
calc = Calculator(records=Records(), policy = Policy())
calc.advance_to_year(2014)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [100]:
# new calculator
growf = Growfactors('../taxdata/stage1/growfactors.csv')
rec = Records(puf, gfactors=growf,
              weights=wt, adjust_ratios=adj,
              start_year=2011)
pol = Policy(gfactors=growf, start_year=2011, num_years=14)
calc_new = Calculator(records=rec, policy=pol)
calc_new.advance_to_year(2014)
calc_new.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


### New Distribution Table 

In [101]:
create_distribution_table(calc_new.records, groupby='weighted_deciles',
                          result_type='weighted_sum')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"16,129,231","-49,911,159,980","11,305,684","97,533,652,415","8,861","233,087,917","67,786,943,015","642,748,270","41,944,860","-54,479,433,156",0,0,"41,944,860","6,539,822","72,907,214","2,698,051,710","-2,589,739,458","5,682,897,776","3,093,158,318"
1,"16,129,740","104,765,578,263","13,619,941","116,100,538,963","54,524","481,622,406","83,052,706,050","5,009,063,879","472,518,329","104,446,429,093","12,759","5,676,625","478,194,954","10,376,250","70,092,933","13,743,168,142","-13,205,256,505","14,406,540,420","1,201,283,914"
2,"16,129,552","167,404,651,473","13,206,972","123,899,834,716","322,359","3,442,830,246","107,854,657,542","24,977,036,357","2,441,549,270","164,997,974,121","29,278","25,793,676","2,467,342,946","349,806,271","101,892,033","26,059,440,944","-23,840,012,236","23,411,380,902","-428,631,333"
3,"16,129,504","260,160,354,647","14,217,052","126,629,762,321","761,920","9,922,608,835","112,484,797,970","71,457,295,454","7,531,098,634","253,797,238,952","14,832","10,551,290","7,541,649,924","1,457,387,204","200,176,715","23,445,808,367","-17,161,368,932","35,465,759,301","18,304,390,369"
4,"16,128,810","375,827,812,248","13,932,164","127,941,410,420","1,713,843","22,558,134,655","117,256,263,390","147,683,483,858","17,073,308,780","361,557,615,494","20,925","46,085,492","17,119,394,273","3,471,442,471","292,770,625","17,336,150,327","-3,395,427,901","48,454,136,218","45,058,708,317"
5,"16,130,563","538,816,769,736","12,882,519","122,569,448,940","3,160,279","48,163,224,449","119,531,700,471","273,025,835,940","33,469,641,341","509,009,321,773","17,994","32,561,428","33,502,202,768","5,621,901,761","389,602,586","8,655,056,626","19,614,846,968","66,393,180,387","86,008,027,355"
6,"16,130,166","751,651,584,306","10,974,087","115,146,346,856","5,120,086","85,900,560,172","131,100,262,358","428,657,023,500","55,299,867,807","698,794,346,772","9,749","35,965,590","55,335,833,396","8,256,004,995","614,301,995","2,957,885,514","44,736,244,882","88,733,251,421","133,469,496,302"
7,"16,129,312","1,093,410,129,928","8,634,853","97,801,958,202","7,489,395","140,071,234,525","146,093,975,016","711,241,490,530","100,854,756,104","1,011,109,876,894","29,463","56,993,861","100,911,749,965","11,325,987,608","954,904,610","1,334,598,494","89,206,068,473","126,243,853,335","215,449,921,809"
8,"16,129,969","1,636,513,956,097","5,471,183","67,031,176,644","10,656,121","233,226,674,814","168,817,157,199","1,168,481,089,654","178,241,406,249","1,504,809,841,682","119,805","247,027,311","178,488,433,561","14,185,363,870","1,508,619,748","1,452,760,002","164,358,929,437","200,693,019,381","365,051,948,818"
9,"16,129,753","4,950,981,616,724","1,808,472","22,272,117,859","14,311,735","614,359,537,265","160,971,180,372","4,158,470,560,204","1,033,109,235,367","4,674,682,041,547","3,985,539","33,703,919,971","1,066,813,155,338","26,258,488,080","29,452,538,124","868,256,058","1,069,138,949,324","348,881,085,870","1,418,020,035,194"


### Current Distribution Table

In [102]:
create_distribution_table(calc.records, groupby='weighted_deciles',
                          result_type='weighted_sum')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"16,222,114","-117,642,730,397","11,094,216","96,752,559,176","6,150","288,416,463","67,077,228,183","707,132,509","58,326,738","-123,301,156,818",0,0,"58,326,738","6,534,866","104,114,514","2,284,974,248","-2,129,067,863","5,733,573,002","3,604,505,139"
1,"16,220,773","97,580,036,362","13,627,903","114,461,983,569","48,948","470,314,529","79,698,496,354","4,249,129,344","377,930,779","97,159,272,826","20,950","6,253,106","384,183,885","5,960,746","65,791,448","12,562,769,796","-12,118,755,208","13,318,269,683","1,199,514,475"
2,"16,223,499","163,827,095,125","13,243,287","123,426,363,072","279,753","2,913,092,844","105,332,431,878","24,036,354,174","2,354,677,750","161,806,454,526","27,189","13,887,837","2,368,565,587","313,771,658","97,248,587","26,763,187,081","-24,611,144,564","23,161,721,038","-1,449,423,526"
3,"16,222,202","245,272,919,379","14,169,419","126,314,754,873","683,754","8,892,476,658","110,668,233,920","64,748,499,687","6,811,152,285","239,558,900,407","31,485","28,990,830","6,840,143,115","1,328,461,112","206,490,156","23,677,789,679","-17,959,617,520","33,072,495,795","15,112,878,275"
4,"16,223,284","364,054,702,131","14,142,144","128,509,105,833","1,516,658","20,886,584,071","116,603,561,906","140,621,774,305","16,213,987,830","351,180,554,668","18,616","37,936,750","16,251,924,580","3,522,446,457","271,705,398","17,305,696,557","-4,304,513,035","46,731,627,548","42,427,114,513"
5,"16,222,555","528,263,202,071","13,206,631","124,699,660,349","2,863,096","43,161,694,001","119,056,478,172","268,185,132,150","32,723,181,029","502,672,002,166","15,840","38,879,146","32,762,060,176","5,877,778,155","424,731,110","8,886,777,131","18,422,235,999","64,330,473,025","82,752,709,024"
6,"16,222,223","753,980,159,239","11,339,850","114,639,366,799","4,849,894","81,691,267,946","125,467,524,211","441,941,009,969","57,961,249,369","705,123,893,445","10,617","31,844,100","57,993,093,469","8,203,321,547","682,348,690","2,662,975,603","47,809,145,009","86,561,378,986","134,370,523,995"
7,"16,222,702","1,099,608,620,029","9,028,824","98,667,637,348","7,185,021","132,308,581,642","139,089,019,394","731,605,500,376","105,755,595,393","1,025,663,100,136","46,267","76,698,684","105,832,294,078","11,040,474,218","1,001,666,943","1,292,077,528","94,501,409,274","125,044,925,483","219,546,334,756"
8,"16,222,104","1,668,257,638,581","5,871,767","69,882,644,295","10,349,258","222,733,967,624","164,524,641,207","1,212,247,826,461","189,826,035,295","1,550,047,600,623","184,541","295,056,578","190,121,091,873","14,427,205,045","1,617,792,894","1,544,499,976","175,767,179,745","202,436,924,101","378,204,103,847"
9,"16,223,164","5,218,313,951,446","2,019,136","24,281,286,951","14,190,613","591,800,540,343","155,421,760,341","4,453,457,287,124","1,122,072,416,954","4,975,981,120,241","4,448,053","38,148,425,391","1,160,220,842,344","29,020,977,897","34,686,229,698","884,181,608","1,165,001,912,538","356,209,808,064","1,521,211,720,602"


### Diagnostic Table Comparison

In [103]:
new_diag = create_diagnostic_table(calc_new)
diag = create_diagnostic_table(calc)  # Current PUF

In [104]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['New'] = new_diag[2014]
diag_data['Current'] = diag[2014]
diag_data['% Change'] = ((new_diag[2014] / diag[2014]) - 1) * 100
diag_data['New - SOI'] = map(abs, new_diag[2014] - diag_data['SOI'])
diag_data['Current - SOI'] = map(abs, diag[2014] - diag_data['SOI'])
diag_data

,SOI,New,Current,% Change,New - SOI,Current - SOI
Returns (#m),148.6,161.3,162.2,-0.6,12.7,13.6
AGI ($b),"9,771.0","9,829.6","10,021.5",-1.9,58.6,250.5
Itemizers (#m),44.0,43.6,42.0,3.9,0.4,2.0
Itemized Deduction ($b),"1,206.7","1,158.4","1,105.2",4.8,48.3,101.5
Standard Deduction Filers (#m),117.4,106.1,107.7,-1.6,11.3,9.7
Standard Deduction ($b),876.2,926.4,925.2,0.1,50.2,49.0
Personal Exemption ($b),"1,121.6","1,149.8","1,114.0",3.2,28.2,7.6
Taxable Income ($b),"6,997.9","6,989.6","7,341.8",-4.8,8.3,343.9
Regular Tax ($b),nan,"1,428.5","1,534.2",-6.9,nan,nan
AMT Income ($b),nan,"9,228.7","9,485.9",-2.7,nan,nan


### Income Levels 

In [105]:
inc_dict = OrderedDict()
inc_dict['New'] = [] 
inc_dict['Current'] = []
inc_dict['SOI'] = []
inc_dict['Pct Diff'] = []
inc_dict['New - SOI'] = []
inc_dict['Current - SOI'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
# Wage and salary
was_new = (calc_new.records.e00200 * calc_new.records.s006).sum()
inc_dict['New'].append(was_new)
was = (calc.records.e00200 * calc.records.s006).sum()
inc_dict['Current'].append(was)
soi_was = soi_income['WAS'].sum()
inc_dict['SOI'].append(soi_was)
new_was_diff = abs(was_new - soi_was)
inc_dict['New - SOI'].append(new_was_diff)
curr_was_diff = abs(was - soi_was)
inc_dict['Current - SOI'].append(curr_was_diff)
inc_dict['Pct Diff'].append((was_diff / was) * 100)
# Interest income
int_new = (calc_new.records.e00300 * calc_new.records.s006).sum()
inc_dict['New'].append(int_new)
int_ = (calc.records.e00300 * calc.records.s006).sum()
inc_dict['Current'].append(int_)
int_soi = soi_income['INT'].sum()
inc_dict['SOI'].append(int_soi)
new_int_diff = abs(int_new - int_soi)
inc_dict['New - SOI'].append(new_int_diff)
curr_int_diff = abs(int_ - int_soi)
inc_dict['Current - SOI'].append(curr_int_diff)
inc_dict['Pct Diff'].append((int_diff / int_) * 100)
# Ordinary dividends
odiv_new = (calc_new.records.e00600 * calc_new.records.s006).sum()
inc_dict['New'].append(odiv_new)
odiv = (calc.records.e00600 * calc.records.s006).sum()
inc_dict['Current'].append(odiv)
odiv_soi = soi_income['ODIV'].sum()
inc_dict['SOI'].append(odiv_soi)
new_odiv_diff = abs(odiv_new - odiv_soi)
inc_dict['New - SOI'].append(new_odiv_diff)
curr_odiv_diff = abs(odiv - odiv_soi)
inc_dict['Current - SOI'].append(curr_odiv_diff)
inc_dict['Pct Diff'].append((odiv_diff / odiv) * 100)
# Qualified dividends
qdiv_new = (calc_new.records.e00650 * calc_new.records.s006).sum()
inc_dict['New'].append(qdiv_new)
qdiv = (calc.records.e00650 * calc.records.s006).sum()
inc_dict['Current'].append(qdiv)
qdiv_soi = soi_income['QDIV'].sum()
inc_dict['SOI'].append(qdiv_soi)
new_qdiv_diff = abs(qdiv_new - qdiv_soi)
inc_dict['New - SOI'].append(new_qdiv_diff)
curr_qdiv_diff = abs(qdiv - qdiv_soi)
inc_dict['Current - SOI'].append(curr_qdiv_diff)
inc_dict['Pct Diff'].append((qdiv_diff / qdiv) * 100)

biz_new = (calc_new.records.e00900 * calc_new.records.s006).sum()
inc_dict['New'].append(biz_new)
biz = (calc.records.e00900 * calc.records.s006).sum()
inc_dict['Current'].append(biz)
biz_soi = soi_income['BIZ'].sum()
inc_dict['SOI'].append(biz_soi)
new_biz_diff = abs(biz_new - biz_soi)
inc_dict['New - SOI'].append(new_biz_diff)
curr_biz_dif = abs(biz - biz_soi)
inc_dict['Current - SOI'].append(curr_biz_dif)
inc_dict['Pct Diff'].append((biz_diff / biz) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
print "'Pct Diff' represents the percentage change between the current and new sums"
inc_df

'Pct Diff' represents the percentage change between the current and new sums


,New,Current,SOI,Pct Diff,New - SOI,Current - SOI
WAS,"6,784,980,284,114.8","6,820,912,371,695.9","6,784,942,966,000.0",-0.5,"37,318,114.8","35,969,405,695.9"
Taxable Interest,"104,562,327,635.2","97,246,196,658.4","93,894,281,000.0",7.5,"10,668,046,635.2","3,351,915,658.4"
Ordinary Dividends,"254,707,967,594.8","270,137,086,776.2","254,702,232,000.0",-5.7,"5,735,594.8","15,434,854,776.2"
Qualified Dividends,"192,856,835,660.6","198,124,994,913.5","192,447,708,000.0",-2.7,"409,127,660.6","5,677,286,913.5"
Business Income,"317,256,458,442.1","309,444,223,964.1","317,258,765,000.0",2.5,"2,306,557.9","7,814,541,035.9"


### Distribution of Income Variables

In [106]:
# Generate data for distribution plots
new_dist = pd.DataFrame()
cur_dist = pd.DataFrame()
new_was = distribution(calc_new.records.e00200, calc_new.records.s006, calc_new.records.c00100)
cur_was = distribution(calc.records.e00200, calc.records.s006, calc.records.c00100)
new_int = distribution(calc_new.records.e00300, calc_new.records.s006, calc_new.records.c00100)
cur_int = distribution(calc.records.e00300, calc.records.s006, calc.records.c00100)
new_odiv = distribution(calc_new.records.e00600, calc_new.records.s006, calc_new.records.c00100)
cur_odiv = distribution(calc.records.e00600, calc.records.s006, calc.records.c00100)
new_qdiv = distribution(calc_new.records.e00650, calc_new.records.s006, calc_new.records.c00100)
cur_qdiv = distribution(calc.records.e00650, calc.records.s006, calc.records.c00100)
new_biz = distribution(calc_new.records.e00900, calc_new.records.s006, calc_new.records.c00100)
cur_biz = distribution(calc.records.e00900, calc.records.s006, calc.records.c00100)
new_dist['WAS'] = new_was[1]
cur_dist['WAS'] = cur_was[1]
new_dist['INT'] = new_int[1]
cur_dist['INT'] = cur_int[1]
new_dist['ODIV'] = new_odiv[1]
cur_dist['ODIV'] = cur_odiv[1]
new_dist['QDIV'] = new_qdiv[1]
cur_dist['QDIV'] = cur_qdiv[1]
new_dist['BIZ'] = new_biz[1]
cur_dist['BIZ'] = cur_biz[1]
new_dist['AGI Bin'] = index_list()
cur_dist['AGI Bin'] = index_list()
new_dist['label'] = 'New'
cur_dist['label'] = 'Current'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum()) * 100
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([new_dist, cur_dist, soi_dist])
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [107]:
show(column(scatter_list))

In [108]:
new_tot = pd.DataFrame()
cur_tot = pd.DataFrame()
new_tot['WAS'] = new_was[0]
cur_tot['WAS'] = cur_was[0]
new_tot['INT'] = new_int[0]
cur_tot['INT'] = cur_int[0]
new_tot['ODIV'] = new_odiv[0]
cur_tot['ODIV'] = cur_odiv[0]
new_tot['QDIV'] = new_qdiv[0]
cur_tot['QDIV'] = cur_qdiv[0]
new_tot['BIZ'] = new_biz[0]
cur_tot['BIZ'] = cur_biz[0]
new_tot['AGI Bin'] = index_list()
cur_tot['AGI Bin'] = index_list()
new_tot['label'] = 'New'
cur_tot['label'] = 'Current'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([new_tot, cur_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
total_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(total_data, x='AGI Bin', y=item[0], color='label', ylabel='Total',
                      title=title, tooltips=[('Total', '@{}'.format(item[0]))])
    total_list.append(scatter)

In [109]:
show(column(total_list))

### Itemized Deduction Amounts

In [110]:
deductions_new = {'Medical Expenses': (calc_new.records.e17500[calc_new.records.c04470 > 0] *
                                        calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'State and Local Taxes':  (calc_new.records.e18400[calc_new.records.c04470 > 0] *
                                              calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Real Estate Taxes':  (calc_new.records.e18500[calc_new.records.c04470 > 0] *
                                          calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Interest Paid':  (calc_new.records.e19200[calc_new.records.c04470 > 0] *
                                       calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Charitable Cash Contributions': (calc_new.records.e19800[calc_new.records.c04470 > 0] *
                                                     calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Charitable Non-Cash Contributions': (calc_new.records.e20100[calc_new.records.c04470 > 0] *
                                                         calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Total Misc. Expenses':  (calc_new.records.e20400[calc_new.records.c04470 > 0] *
                                             calc_new.records.s006[calc_new.records.c04470 > 0]).sum(),
                   'Net Casualty or Loss': (calc_new.records.g20500[calc_new.records.c04470 > 0] *
                                            calc_new.records.s006[calc_new.records.c04470 > 0]).sum()}
ded_new_df = pd.DataFrame.from_dict(deductions_new, 'index')
ded_new_df.columns = ['Total']
ded_new_df['source'] = 'New'

deductions = {'Medical Expenses': (calc.records.e17500[calc.records.c04470 > 0] *
                                   calc.records.s006[calc.records.c04470 > 0]).sum(),
              'State and Local Taxes':  (calc.records.e18400[calc.records.c04470 > 0] *
                                         calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Real Estate Taxes':  (calc.records.e18500[calc.records.c04470 > 0] *
                                     calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Interest Paid':  (calc.records.e19200[calc.records.c04470 > 0] *
                                 calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Cash Contributions': (calc.records.e19800[calc.records.c04470 > 0] *
                                                calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Non-Cash Contributions': (calc.records.e20100[calc.records.c04470 > 0] *
                                                    calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Total Misc. Expenses':  (calc.records.e20400[calc.records.c04470 > 0] *
                                        calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Net Casualty or Loss': (calc.records.g20500[calc.records.c04470 > 0] *
                                       calc.records.s006[calc.records.c04470 > 0]).sum()}
ded_df = pd.DataFrame.from_dict(deductions, 'index')
ded_df.columns = ['Total']
ded_df['source'] = 'Current'
soi_deductions['source'] = 'SOI'

ded_full_df = pd.concat([ded_new_df, ded_df, soi_deductions])
ded_full_df['index'] = ded_full_df.index

In [111]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height{0.2f}'),
                                                            ('Data', '@source')])
show(ded_bar)

In [117]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference: New'] = ded_new_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: New'] = 100 * ded_error_df['Difference: New'] / soi_deductions['Total']
ded_error_df['Difference: Current'] = ded_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: Current'] = 100 * ded_error_df['Difference: Current'] / soi_deductions['Total']
print 'Error in Itemized Deductions Relative to SOI Totals'
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference: New,% Difference: New,Difference: Current,% Difference: Current
Charitable Cash Contributions,"-5,987,233,855.7",-3.9,"-4,703,427,583.1",-3.0
Charitable Non-Cash Contributions,"-30,160,002,722.4",-46.2,"-31,555,102,263.5",-48.3
Interest Paid,"60,086,643,754.4",19.5,"-7,489,448,194.8",-2.4
Medical Expenses,"-17,629,204,361.4",-13.7,"-9,779,156,210.6",-7.6
Net Casualty or Loss,"1,757,990,563.6",79.8,"2,251,393,716.6",102.1
Real Estate Taxes,"11,388,573,128.2",6.3,"17,247,882,146.1",9.5
State and Local Taxes,"-18,564,889,867.9",-5.7,"-16,248,434,290.2",-4.9
Total Misc. Expenses,"1,167,649,238.6",0.9,"3,957,677,605.4",3.1


In [113]:
# Create DataFrame subset of calculator records for the new weights
item_df_new = pd.DataFrame({'e00200': calc_new.records.e00200,
                             's006': calc_new.records.s006,
                             'c04470': calc_new.records.c04470})
item_df_new['itemizer'] = np.where(calc_new.records.c04470 > 0, 1, 0)
item_participation_rt_new = percentile(item_df_new, 'itemizer', 100, 'e00200', 's006')
# Create DataFrame subset for current weights
item_df = pd.DataFrame({'e00200': calc.records.e00200,
                        's006': calc.records.s006,
                        'c04470': calc.records.c04470})
item_df['itemizer'] = np.where(calc.records.c04470 > 0, 1, 0)
item_participation_rt = percentile(item_df, 'itemizer', 100, 'e00200', 's006')
item_index = item_participation_rt_new.index

In [114]:
hover_ded = HoverTool(tooltips=[('Percentile', '$index'), ('Itemizing Rate', '$x')])
f = figure(title='Itemizing Rate by Wage Percentile',
           x_axis_label='Wage Percentile', tools=[hover_ded, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_participation_rt_new, legend='new')
f.line(item_index, item_participation_rt, legend='Current', color='red')
f.legend.location = 'top_left'
show(f)

In [115]:
# Average Itemized Deduction by percentile
itemizers_new = copy.deepcopy(item_df_new[item_df_new['c04470'] > 0])
item_mean_new = percentile(itemizers_new, 'c04470', 100, 'e00200', 's006')
itemizers = copy.deepcopy(item_df[item_df['c04470'] > 0])
item_mean = percentile(itemizers, 'c04470', 100, 'e00200', 's006')

In [116]:
hover = HoverTool(tooltips=[('Percentile', '$index'), ('Mean', '$y')])
f = figure(title='Mean Itemized Deduction Total Among Itemizers',
           x_axis_label='Wage Percentile', tools=[hover, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_mean_new, legend='new')
f.line(item_index, item_mean, legend='Current', color='red')
f.legend.location = 'top_left'
show(f)